In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/CuongNXV/Voice_Processing

fatal: destination path 'Voice_Processing' already exists and is not an empty directory.


In [3]:
import pandas as pd
import csv
#chuuyển label sang chuỗi 0-1
from sklearn.preprocessing import LabelEncoder 
import numpy as np
import os
import librosa
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

Using TensorFlow backend.


In [0]:
train = pd.read_csv('/content/Voice_Processing/data/Train/train.csv')
test = pd.read_csv('/content/Voice_Processing/data/Test/test.csv')
val = pd.read_csv('/content/Voice_Processing/data/Validation/validation.csv')
#train.head(941)

In [0]:
label_data = ['air_conditioner',
'car_horn',
'children_playing',
'dog_bark',
'drilling',
'engine_idling',
'gun_shot',
'jackhammer',
'siren',
'street_music']

In [0]:
def getFeature_Train(row):
   filename = '/content/Voice_Processing/data/Train/Train/' + str(row.ID) + '.wav'
   data, sample_rate = librosa.load(filename, res_type='kaiser_fast')
   mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40).T,axis=0) 
   feature = mfccs
   label = row.Class
   print(filename)
   return pd.Series([feature, label])


In [0]:
def getFeature_Val(row):
   filename = '/content/Voice_Processing/data/Validation/Validation/' + str(row.ID) + '.wav'
   data, sample_rate = librosa.load(filename, res_type='kaiser_fast')
   mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40).T,axis=0) 
   feature = mfccs
   label = row.Class
   print(filename)
   return pd.Series([feature, label])



In [0]:
def getFeature_Test(row):
   filename = '/content/Voice_Processing/data/Test/Test/' + str(row.ID) + '.wav'
   data, sample_rate = librosa.load(filename, res_type='kaiser_fast')
   mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40).T,axis=0) 
   feature = mfccs
   label = row.Class
   print(filename)
   return pd.Series([feature, label])



In [15]:
temp_train = train.apply(getFeature_Train, axis=1)
temp_train.columns = ['feature', 'label']

/content/Voice_Processing/data/Train/Train/0.wav
/content/Voice_Processing/data/Train/Train/0.wav
/content/Voice_Processing/data/Train/Train/1.wav
/content/Voice_Processing/data/Train/Train/2.wav
/content/Voice_Processing/data/Train/Train/3.wav
/content/Voice_Processing/data/Train/Train/4.wav
/content/Voice_Processing/data/Train/Train/6.wav
/content/Voice_Processing/data/Train/Train/10.wav
/content/Voice_Processing/data/Train/Train/11.wav
/content/Voice_Processing/data/Train/Train/12.wav
/content/Voice_Processing/data/Train/Train/15.wav
/content/Voice_Processing/data/Train/Train/17.wav
/content/Voice_Processing/data/Train/Train/18.wav
/content/Voice_Processing/data/Train/Train/19.wav
/content/Voice_Processing/data/Train/Train/20.wav
/content/Voice_Processing/data/Train/Train/22.wav
/content/Voice_Processing/data/Train/Train/24.wav
/content/Voice_Processing/data/Train/Train/26.wav
/content/Voice_Processing/data/Train/Train/27.wav
/content/Voice_Processing/data/Train/Train/32.wav
/conten

In [16]:
temp_val = val.apply(getFeature_Val, axis=1)
temp_val.columns = ['feature', 'label']

/content/Voice_Processing/data/Validation/Validation/2415.wav
/content/Voice_Processing/data/Validation/Validation/2415.wav
/content/Voice_Processing/data/Validation/Validation/2416.wav
/content/Voice_Processing/data/Validation/Validation/2418.wav
/content/Voice_Processing/data/Validation/Validation/2419.wav
/content/Voice_Processing/data/Validation/Validation/2420.wav
/content/Voice_Processing/data/Validation/Validation/2421.wav
/content/Voice_Processing/data/Validation/Validation/2424.wav
/content/Voice_Processing/data/Validation/Validation/2425.wav
/content/Voice_Processing/data/Validation/Validation/2427.wav
/content/Voice_Processing/data/Validation/Validation/2428.wav
/content/Voice_Processing/data/Validation/Validation/2430.wav
/content/Voice_Processing/data/Validation/Validation/2432.wav
/content/Voice_Processing/data/Validation/Validation/2433.wav
/content/Voice_Processing/data/Validation/Validation/2438.wav
/content/Voice_Processing/data/Validation/Validation/2439.wav
/content

In [17]:
temp_test = test.apply(getFeature_Test, axis=1)
temp_test.columns = ['feature', 'label']

/content/Voice_Processing/data/Test/Test/2728.wav
/content/Voice_Processing/data/Test/Test/2728.wav
/content/Voice_Processing/data/Test/Test/2730.wav
/content/Voice_Processing/data/Test/Test/2732.wav
/content/Voice_Processing/data/Test/Test/2733.wav
/content/Voice_Processing/data/Test/Test/2736.wav
/content/Voice_Processing/data/Test/Test/2738.wav
/content/Voice_Processing/data/Test/Test/2739.wav
/content/Voice_Processing/data/Test/Test/2740.wav
/content/Voice_Processing/data/Test/Test/2741.wav
/content/Voice_Processing/data/Test/Test/2744.wav
/content/Voice_Processing/data/Test/Test/2746.wav
/content/Voice_Processing/data/Test/Test/2747.wav
/content/Voice_Processing/data/Test/Test/2750.wav
/content/Voice_Processing/data/Test/Test/2751.wav
/content/Voice_Processing/data/Test/Test/2752.wav
/content/Voice_Processing/data/Test/Test/2754.wav
/content/Voice_Processing/data/Test/Test/2756.wav
/content/Voice_Processing/data/Test/Test/2757.wav
/content/Voice_Processing/data/Test/Test/2760.wav


In [0]:
X = np.array(temp_train.feature.tolist())
y = np.array(temp_train.label.tolist())
lb = LabelEncoder()
y = np_utils.to_categorical(lb.fit_transform(y))

In [0]:
val_X = np.array(temp_val.feature.tolist())
val_y = np.array(temp_val.label.tolist())
lb = LabelEncoder()
val_y = np_utils.to_categorical(lb.fit_transform(val_y))

In [0]:
test_X = np.array(temp_test.feature.tolist())
test_y = np.array(temp_test.label.tolist())
lb = LabelEncoder()
test_y = np_utils.to_categorical(lb.fit_transform(test_y))

In [24]:
num_labels = y.shape[1]

# Model DNN
model = Sequential()

model.add(Dense(256, input_shape=(X.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [26]:
model.fit(X, y, batch_size=128, epochs=100, validation_data=(val_X, val_y))

Train on 1500 samples, validate on 200 samples
Epoch 1/100
1500/1500 [==============================] - 0s 55us/step - loss: 0.7881 - acc: 0.7460 - val_loss: 0.6668 - val_acc: 0.7800
Epoch 2/100
1500/1500 [==============================] - 0s 47us/step - loss: 0.7463 - acc: 0.7453 - val_loss: 0.6657 - val_acc: 0.7700
Epoch 3/100
1500/1500 [==============================] - 0s 43us/step - loss: 0.7548 - acc: 0.7367 - val_loss: 0.6701 - val_acc: 0.7750
Epoch 4/100
1500/1500 [==============================] - 0s 44us/step - loss: 0.7725 - acc: 0.7380 - val_loss: 0.6691 - val_acc: 0.7800
Epoch 5/100
1500/1500 [==============================] - 0s 40us/step - loss: 0.7538 - acc: 0.7260 - val_loss: 0.6614 - val_acc: 0.7800
Epoch 6/100
1500/1500 [==============================] - 0s 43us/step - loss: 0.7150 - acc: 0.7580 - val_loss: 0.6502 - val_acc: 0.7850
Epoch 7/100
1500/1500 [==============================] - 0s 42us/step - loss: 0.7486 - acc: 0.7573 - val_loss: 0.6519 - val_acc: 0.7800
E

In [27]:
results = model.evaluate(test_X, test_y)
print(results)
print(model.metrics_names)

500/500 [==============================] - 0s 65us/step
[0.5754783601760864, 0.8160000004768372]
['loss', 'acc']


In [0]:
import pickle
filename = '/content/drive/My Drive/XuLyTiengNoi/model/train_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [0]:
def random_sound():
    path = '/content/drive/My Drive/XuLyTiengNoi/test/Test'
    files = os.listdir(path)
    index = random.randrange(0, len(files))
    print('Sourse: ' + str(files[index]))
    filename = '/content/drive/My Drive/XuLyTiengNoi/test/Test/' + str(files[index])
    return filename

In [0]:
def feature_test(filename):
    data, sample_rate = librosa.load(filename, res_type='kaiser_fast')
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40).T,axis=0) 
    feature = np.array(mfccs)
    return feature

In [0]:
import matplotlib.pyplot as plt
import librosa.display
def feature_sound(filename):
    data, sampling_rate = librosa.load(filename, sr = None)
    plt.figure(figsize=(15, 5))
    librosa.display.waveplot(data, sampling_rate)

In [0]:

filename_test = random_sound()
print(filename_test)
test_X = feature_test(filename_test)
feature_sound(filename_test)

In [0]:
from IPython.display import Audio
Audio(filename_test)

In [0]:
test_X = test_X.reshape(1, 40)
predictions = model.predict(test_X)
index = np.argmax(predictions)
print(predictions)
print(np.max(predictions))
if np.max(predictions) < 0.25:
  print('undefine')
else:
  print(label_data[index])
